In [44]:
import re
import spacy
import textract
import os
from spacy.matcher import Matcher
import glob
import os
import json
import random

In [39]:
class parse:
    def __init__(self, file_path):
        self.file_path = file_path
        self.resume = textract.process(self.file_path).decode()
        if type(self.resume) == str:
            self.resumes = [self.resume]
        self.nlp = spacy.load('en_core_web_sm')
        self.matcher = Matcher(self.nlp.vocab)

    def clean_text(self):
        clean_resume = []
        for resume in self.resumes:
            clean_resume.append(re.sub(r'[^\w\s]', '', resume))
        return clean_resume

    def extract_phone_number(self):
        ph_list = []
        for resume in self.resumes:
            items = re.finditer(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}', resume)
            ph_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            ph_list.append(ph_span)
        return ph_list
    
    def extract_email(self):
        email_list = []
        for resume in self.resumes:
            items = re.finditer(r'[\w\.-]+@[\w\.-]+', resume)
            em_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            email_list.append(em_span)
        return email_list
       
    def extract_dob(self):
        dob_list = []
        for resume in self.resumes:
            result2 = re.finditer(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}", resume) # DD-MM-YYYY
            dob_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in result2]
            dob_list.append(dob_span)
        return dob_list

    def extract_name(self):
        name_list = []
        for resume in self.resumes:
            doc = self.nlp(resume)
            pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
            self.matcher.add('NAME', [pattern])
            matches = self.matcher(doc)
            for match_id, start, end in matches:
                span = doc[start:end]
        return span.text

    def extract_location(self):
        location_list = []
        for resume in self.resumes:
            doc = self.nlp(resume)
            for ent in doc.ents:
                if ent.label_ in ['GPE', 'LOC']:
                    ls = list(ent.text)
            string = ''.join(ls)
            location_list.append(string)
        return location_list
    
    def output(self, *args):
        return {
            'name': self.extract_name(),
            'phone number': self.extract_phone_number(),
            'e-mail': self.extract_email(),
            'DoB': self.extract_dob(),
            'location': self.extract_location()
        }
    def output_to_json(self, *args):
        s = {
            'name': self.extract_name(),
            'phone number': self.extract_phone_number(),
            'e-mail': self.extract_email(),
            'DoB': self.extract_dob(),
            'location': self.extract_location()
        }
        with open('sample6.json','w') as outfile:
            json_obj = json.dumps(s)
            outfile.write(json_obj)
        return "File Created"

In [40]:
file_path = 'Resume1 UI.docx'

In [41]:
parser = parse(file_path)

In [42]:
parser.output()

{'name': 'CLOUD ARCHITECT',
 'phone number': [[('123-456-7890', (27, 39))]],
 'e-mail': [[('xyz@gmail.com', (48, 61)),
   ('eymemathew@gmail.com', (13086, 13106)),
   ('fyan@unr.edu', (13190, 13202)),
   ('SalmanKaleemM@Virtusa.com', (13269, 13294))]],
 'DoB': [[]],
 'location': ['India']}

In [45]:
parser.output_to_json()

'File Created'